In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict
from functools import reduce

/nfs/homedirs/fuchsgru/miniconda3/lib/python3.9/site-packages/seml/database.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
collection_name = 'week9_cora_full'
collection = seml.database.get_collection(collection_name)
experiments = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]
for ex in experiments:
    with open(ex['result']) as f:
        ex['result'] = json.load(f)

print(f'Number of finished experiments : {len(experiments)}')

Number of finished experiments : 14


In [3]:
def get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True):
    exs = [
        r for r in experiments if r['config']['model']['residual'] == residual and r['config']['model']['use_spectral_norm'] == spectral_norm and r['config']['data']['train_labels_remove_other'] == train_labels_remove_other
    ]
    return exs[0]

In [4]:
experiment = get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True)

In [5]:
print([k for k in experiment['result'].keys() if 'acc' in k])

['val_accuracy-val-train-labels-0', 'val_accuracy-val-reduced-0', 'accuracy_val-reduced', 'accuracy_id_val-reduced', 'accuracy_ood_val-reduced', 'accuracy_val-reduced_no-edges', 'accuracy_id_val-reduced_no-edges', 'accuracy_ood_val-reduced_no-edges', 'accuracy_val_loc', 'accuracy_id_val_loc', 'accuracy_ood_val_loc', 'accuracy_val_loc-no-edges', 'accuracy_id_val_loc-no-edges', 'accuracy_ood_val_loc-no-edges', 'accuracy_val-reduced-bernoulli_bernoulli', 'accuracy_id_val-reduced-bernoulli_bernoulli', 'accuracy_ood_val-reduced-bernoulli_bernoulli', 'accuracy_val-reduced-bernoulli_bernoulli-no-edges', 'accuracy_id_val-reduced-bernoulli_bernoulli-no-edges', 'accuracy_ood_val-reduced-bernoulli_bernoulli-no-edges', 'accuracy_val-reduced-normal_normal', 'accuracy_id_val-reduced-normal_normal', 'accuracy_ood_val-reduced-normal_normal', 'accuracy_val-reduced-normal_normal-no-edges', 'accuracy_id_val-reduced-normal_normal-no-edges', 'accuracy_ood_val-reduced-normal_normal-no-edges']


In [6]:

d = {}
for experiment in experiments:
    residual = experiment['config']['model']['residual']
    spectral_norm = experiment['config']['model']['use_spectral_norm']
    remove_other = experiment['config']['data']['train_labels_remove_other']
    if any('Operating_Systems' in label for label in experiment['config']['data']['val_labels']):
        ood = 'os'
    else:
        ood = 'ai'
    
    prefix = (ood, remove_other, residual, spectral_norm)
    
    for k, v in experiment['result'].items():
        mean, std = np.array(v).mean(), np.array(v).std()
        med = np.median(np.array(v))
        if 'auroc' in k:
            if 'no-edges' in k:
                no_edges = True
                k = k.replace('_no-edges', '')
                k = k.replace('-no-edges', '')
            else:
                no_edges = False
            
            k_tokens = k.split('_')
            k, ood_type = '_'.join(k_tokens[:-1]), k_tokens[-1]
            prefix_edges = prefix + (no_edges, ood_type,)
                
            if not 'no' in k and ':' in k:
                continue
            d[prefix_edges + ('AUC-ROC', k.replace('auroc_', '').replace('-', ' '), 'mean')] = [mean]
            d[prefix_edges + ('AUC-ROC', k.replace('auroc_', '').replace('-', ' '), 'std')] = [std]
            #d[prefix_edges + ('feature density', k.replace('auroc:', ''), 'median')] = [med]
        elif k.startswith('accuracy'):
            k_tokens = k.split('_')
            if k_tokens[1] in ('id', 'ood'):
                k_tokens = ['_'.join(k_tokens[:2])] + k_tokens[2:]
            if 'no-edges' in k_tokens[-1]:
                k_tokens[-1] = k_tokens[-1].replace('no-edges', '')[:-1]
                no_edges = True
            else:
                no_edges = False
            k_tokens = [token for token in k_tokens if len(token) > 0]
            if len(k_tokens) == 3:
                ood_type = k_tokens[-1]
                k_tokens = k_tokens[:-1]
            else:
                #ood_type = ''
                continue
            name, dataset = k_tokens
            
            d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'mean')] = [mean]
            d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'std')] = [std]
            #d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'med')] = [med]
        elif k.startswith('ece') and False:
            if 'no_edges' in k:
                k = k.replace('no_edges', '')[:-1]
                no_edges = True
            else:
                no_edges = False
            
            k_tokens = k.split('_')
            d[prefix + (no_edges, '', 'ece', '', 'mean')] = [mean]
            d[prefix + (no_edges, '', 'ece', '', 'std')] = [std]
            
            #print(name, dataset, prefix + (no_edges, ood_type,))
            
            
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'mean')] = [mean]
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'std')] = [std]
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'median')] = [med]
                
df = pd.DataFrame(d).T.sort_index()

In [7]:
df

0
ai False False False False bernoulli AUC-ROC  7 mog:no     mean  0.432812
                                                           std   0.065340
                                              gpc full:no  mean  0.455578
                                                           std   0.077607
                                              logit energy mean  0.583401
...                                                                   ...
os True  True  True  True  normal    accuracy              std   0.023602
                                              id           mean  0.762222
                                                           std   0.027353
                                              ood          mean  0.142857
                                                           std   0.092582

[1344 rows x 1 columns]

In [8]:
df = df.reset_index((0, 1, 2, 3, 5, -1))

In [9]:
df.shape

(1344, 7)

In [10]:
df

level_0  level_1  level_2  level_3    level_5  \
False AUC-ROC  7 mog:no          ai    False    False    False  bernoulli   
               7 mog:no          ai    False    False    False  bernoulli   
               gpc full:no       ai    False    False    False  bernoulli   
               gpc full:no       ai    False    False    False  bernoulli   
               logit energy      ai    False    False    False  bernoulli   
...                             ...      ...      ...      ...        ...   
True  accuracy                   os     True     True     True     normal   
               id                os     True     True     True     normal   
               id                os     True     True     True     normal   
               ood               os     True     True     True     normal   
               ood               os     True     True     True     normal   

                            level_8         0  
False AUC-ROC  7 mog:no        mean  0.432812  
               7 mog:no         std  0.065340  
               gpc full:no     mean  0.455578  
               gpc full:no      std  0.077607  
               logit energy    mean  0.583401  
...                             ...       ...  
True  accuracy                  std  0.023602  
               id              mean  0.762222  
               id               std  0.027353  
               ood             mean  0.142857  
               ood              std  0.092582  

[1344 rows x 7 columns]

In [11]:
subdfs, names = [], []
for n, g in df.groupby(['level_0', 'level_1', 'level_2', 'level_3', 'level_5', 'level_8']):
    g = pd.DataFrame(g[0])
    g.columns = [n]
    subdfs.append(g)

In [12]:
df_cat = pd.concat(subdfs, axis=1)
idx = pd.MultiIndex.from_tuples(df_cat.columns, names=('OOD Classes', 'Remove OOD', 'Residual', 'Spectral Norm', 'OOD Experiment', 'Stat'))
df_cat.columns = idx
df_cat.index.names = ('Remove-Edges', '', '')
pd.set_option("display.precision", 2)
df_cat = df_cat.T.sort_index().T

In [13]:
exp_sizes = [
    1, # ood data
    1, # remove ood vertices from train
    2, # residual,
    2, # Spectral norm,
    3, # ood type
    2, # stats
]
exp_size = reduce(lambda x, y: x * y, exp_sizes)
exp_size

24

In [14]:


df_cat.iloc[:, 0 * exp_size : 1 * exp_size : 2]

OOD Classes                                           ai                  \
Remove OOD                                         False                   
Residual                                           False                   
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                  4.33e-01  0.67  9.98e-01   
                      gpc full:no               4.56e-01  0.73  9.98e-01   
                      logit energy              5.83e-01  0.83  2.49e-01   
                      max score                 5.75e-01  0.83  4.38e-01   
                      total predictive entropy  5.82e-01  0.84  3.95e-01   
             accuracy                           8.07e-01  0.81  7.59e-01   
                      id                        8.12e-01  0.78  7.98e-01   
                      ood                       7.63e-01   NaN  4.09e-01   
True         AUC-ROC  7 mog:no                  3.17e-04  0.41  1.00e+00   
                      gpc full:no               0.00e+00  0.44  1.00e+00   
                      logit energy              9.96e-01  0.76  5.22e-04   
                      max score                 1.00e+00  0.73  9.68e-02   
                      total predictive entropy  1.00e+00  0.74  8.70e-02   
             accuracy                           7.07e-01  0.76  7.01e-01   
                      id                        7.65e-01  0.71  7.59e-01   
                      ood                       1.83e-01   NaN  1.74e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                      0.39  0.62  9.98e-01   
                      gpc full:no                   0.45  0.76  9.98e-01   
                      logit energy                  0.57  0.70  3.05e-01   
                      max score                     0.58  0.82  3.83e-01   
                      total predictive entropy      0.58  0.79  2.88e-01   
             accuracy                               0.80  0.80  7.52e-01   
                      id                            0.81  0.79  7.90e-01   
                      ood                           0.74   NaN  4.14e-01   
True         AUC-ROC  7 mog:no                      0.00  0.38  1.00e+00   
                      gpc full:no                   0.00  0.48  1.00e+00   
                      logit energy                  0.94  0.77  5.26e-03   
                      max score                     0.99  0.79  3.24e-02   
                      total predictive entropy      0.99  0.80  5.51e-03   
             accuracy                               0.70  0.76  6.96e-01   
                      id                            0.76  0.74  7.58e-01   
                      ood                           0.14   NaN  1.34e-01   

OOD Classes                                                               \
Remove OOD                                         True                    
Residual                                           False                   
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                  

In [15]:

df_cat.iloc[:, 1 * exp_size : 2 * exp_size : 2]

OOD Classes                                           ai                  \
Remove OOD                                         True                    
Residual                                           True                    
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                      0.42  0.63  9.99e-01   
                      gpc full:no                   0.46  0.75  9.99e-01   
                      logit energy                  0.66  0.86  9.36e-02   
                      max score                     0.65  0.87  3.41e-01   
                      total predictive entropy      0.67  0.87  3.03e-01   
             accuracy                               0.82  0.81  7.43e-01   
                      id                            0.82  0.82  8.05e-01   
                      ood                           0.81   NaN  1.89e-01   
True         AUC-ROC  7 mog:no                      0.09  0.53  1.00e+00   
                      gpc full:no                   0.11  0.62  1.00e+00   
                      logit energy                  0.98  0.81  1.86e-03   
                      max score                     1.00  0.79  1.31e-01   
                      total predictive entropy      1.00  0.80  1.17e-01   
             accuracy                               0.70  0.76  6.93e-01   
                      id                            0.76  0.74  7.56e-01   
                      ood                           0.14   NaN  1.26e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                      0.50  0.47  1.00e+00   
                      gpc full:no                   0.66  0.68  1.00e+00   
                      logit energy                  0.65  0.75  6.56e-02   
                      max score                     0.70  0.85  1.07e-01   
                      total predictive entropy      0.71  0.83  3.73e-02   
             accuracy                               0.82  0.82  7.56e-01   
                      id                            0.83  0.83  8.19e-01   
                      ood                           0.74   NaN  1.89e-01   
True         AUC-ROC  7 mog:no                      0.34  0.41  1.00e+00   
                      gpc full:no                   0.67  0.59  1.00e+00   
                      logit energy                  0.86  0.80  2.22e-03   
                      max score                     0.96  0.84  1.42e-02   
                      total predictive entropy      0.97  0.84  5.22e-04   
             accuracy                               0.70  0.77  7.02e-01   
                      id                            0.77  0.76  7.66e-01   
                      ood                           0.13   NaN  1.31e-01   

OOD Classes                                           os                  \
Remove OOD                                         False                   
Residual                                           False                   
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                  

In [16]:

df_cat.iloc[:, 2 * exp_size : 3 * exp_size : 2]

OOD Classes                                           os                  \
Remove OOD                                         False                   
Residual                                           True                    
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                      0.42  0.65  1.00e+00   
                      gpc full:no                   0.42  0.68  9.99e-01   
                      logit energy                  0.65  0.91  7.78e-02   
                      max score                     0.64  0.91  3.05e-01   
                      total predictive entropy      0.66  0.92  2.63e-01   
             accuracy                               0.80  0.80  7.31e-01   
                      id                            0.81  0.77  7.92e-01   
                      ood                           0.73   NaN  1.83e-01   
True         AUC-ROC  7 mog:no                      0.09  0.56  1.00e+00   
                      gpc full:no                   0.09  0.59  1.00e+00   
                      logit energy                  0.98  0.82  2.52e-03   
                      max score                     1.00  0.79  1.13e-01   
                      total predictive entropy      1.00  0.81  1.01e-01   
             accuracy                               0.69  0.74  6.79e-01   
                      id                            0.75  0.69  7.42e-01   
                      ood                           0.16   NaN  1.11e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                      0.45  0.38  1.00e+00   
                      gpc full:no                   0.59  0.56  1.00e+00   
                      logit energy                  0.59  0.73  7.20e-02   
                      max score                     0.67  0.88  9.69e-02   
                      total predictive entropy      0.67  0.86  3.88e-02   
             accuracy                               0.76  0.77  7.07e-01   
                      id                            0.77  0.77  7.63e-01   
                      ood                           0.65   NaN  2.03e-01   
True         AUC-ROC  7 mog:no                      0.32  0.34  1.00e+00   
                      gpc full:no                   0.52  0.48  1.00e+00   
                      logit energy                  0.88  0.84  3.24e-03   
                      max score                     0.98  0.87  2.04e-02   
                      total predictive entropy      0.99  0.88  1.54e-03   
             accuracy                               0.67  0.72  6.67e-01   
                      id                            0.73  0.71  7.24e-01   
                      ood                           0.13   NaN  1.51e-01   

OOD Classes                                                               \
Remove OOD                                         True                    
Residual                                           False                   
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                  

In [17]:

df_cat.iloc[:, 3 * exp_size : 4 * exp_size : 2]

OOD Classes                                           os                  \
Remove OOD                                          True                   
Residual                                            True                   
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                      0.42  0.64  9.99e-01   
                      gpc full:no                   0.46  0.76  9.99e-01   
                      logit energy                  0.66  0.89  9.36e-02   
                      max score                     0.65  0.89  3.41e-01   
                      total predictive entropy      0.67  0.90  3.03e-01   
             accuracy                               0.82  0.81  7.43e-01   
                      id                            0.82  0.82  8.05e-01   
                      ood                           0.81   NaN  1.89e-01   
True         AUC-ROC  7 mog:no                      0.09  0.53  1.00e+00   
                      gpc full:no                   0.11  0.62  1.00e+00   
                      logit energy                  0.98  0.84  1.86e-03   
                      max score                     1.00  0.81  1.31e-01   
                      total predictive entropy      1.00  0.83  1.18e-01   
             accuracy                               0.70  0.76  6.93e-01   
                      id                            0.76  0.74  7.57e-01   
                      ood                           0.14   NaN  1.26e-01   

OOD Classes                                                               
Remove OOD                                                                
Residual                                                                  
Spectral Norm                                      True                   
OOD Experiment                                 bernoulli   loc    normal  
Stat                                                mean  mean      mean  
Remove-Edges                                                              
False        AUC-ROC  7 mog:no                      0.49  0.46  1.00e+00  
                      gpc full:no                   0.67  0.69  1.00e+00  
                      logit energy                  0.65  0.76  6.34e-02  
                      max score                     0.70  0.88  1.07e-01  
                      total predictive entropy      0.71  0.86  3.56e-02  
             accuracy                               0.82  0.82  7.59e-01  
                      id                            0.83  0.82  8.20e-01  
                      ood                           0.74   NaN  2.03e-01  
True         AUC-ROC  7 mog:no                      0.33  0.38  1.00e+00  
                      gpc full:no                   0.67  0.60  1.00e+00  
                      logit energy                  0.87  0.85  2.31e-03  
                      max score                     0.97  0.89  1.63e-02  
                      total predictive entropy      0.97  0.90  8.84e-04  
             accuracy                               0.70  0.76  7.00e-01  
                      id                            0.76  0.75  7.62e-01  
                      ood                           0.13   NaN  1.43e-01